## Foundations of Machine Learning 
### Assignment 1
###### Submitted by - 
Sakshi Badole
CS24MTECH11008

##### Importing Pandas and Numpy

In [490]:
import numpy as np
import pandas as pd

##### Reading data from dataset into a dataframe

In [491]:

red_df = pd.read_csv('wine+quality/winequality-red.csv', sep=';') 
white_df = pd.read_csv('wine+quality/winequality-white.csv', sep=';') 

#merge white and red wine data files
data = red_df.merge(white_df, how='outer')



##### Train-Test-Split: Coded from scratch

In [492]:
#train-test-split

indices_array = np.array(data.index.tolist())
test_size = int(0.2*len(data))

# randomly assigns 20 percent of the indices as the test indices
test_indices = indices_array[np.random.choice(indices_array.shape[0], test_size, replace=False)]

# test data for that the randomly picked indices 
test_df = data.loc[test_indices]

# train dataframe created by dropping the test indices
train_df = data.drop(test_indices)



##### Data Processing

In [493]:
#dataframe to 2D array using .values function

X_train = train_df.drop('quality', axis=1).values

X_test = test_df.drop('quality', axis=1).values
# train and test label(quality) being assigned 
y_train = train_df['quality']
y_test = test_df['quality']

# flatten 2D array to 1D array 
y_data = y_train.values.flatten()
# Assigning new class values i.e 0 for quality values under 7 and 1 for quality values >= 7
y_data = np.where(y_data >= 7 , 1, 0)
y_test = y_test.values.flatten()
y_test = np.where(y_test >= 7 , 1, 0) 


##### Information Gain Function

In [494]:
def calculate_information_gain(X, y, feature_idx, limit, mode='entropy'):
    #parent and weighted child entropy is being calculated for the given feature index
    # returns the information gain i.e. parent - weighted child entropy
    if mode == 'gini':
        parent_gini = calculate_gini(y)
        left_y = y[limit >= X[:, feature_idx] ]
        right_y = y[limit < X[:, feature_idx] ]
        left_gini = calculate_gini(left_y)
        right_gini = calculate_gini(right_y)
        return parent_gini - (len(left_y) / len(y))*left_gini + (len(right_y) / len(y))*right_gini
    else:
        parent_entropy = calculate_entropy(y)
        left_y = y[limit >= X[:, feature_idx] ]
        right_y = y[limit < X[:, feature_idx] ]
        left_entropy = calculate_entropy(left_y)
        right_entropy = calculate_entropy(right_y)
        child_entropy = (len(left_y) / len(y)) * left_entropy + (len(right_y) / len(y)) * right_entropy
        return parent_entropy - child_entropy

##### Entropy Calculation Function

In [495]:
def calculate_entropy(y):
    # calculates the entropy of the entered subtree/tree
    unique, counts = np.unique(y, return_counts=True)
    sum = 0
    y_prob = counts / len(y)

    for p in y_prob:
        if p > 0:
            sum = sum + p * np.log2(p)
    # returns the entropy value using the formula -Summation(p*log(p))
    # here p is the probability of a sample of that class
    return -sum
    # return -np.sum([p * np.log2(p) for p in y_prob if p>0]) 

##### Gini Index (1-Summation(p^2))

In [496]:
def calculate_gini(y):
    unique, counts = np.unique(y, return_counts =True)
    sum=0
    y_prob = counts / len(y)
    for p in y_prob:
        sum = sum + p**2
    return ( 1 - sum )

##### Decision Tree

In [497]:
def decision_tree(X_train, y_train, max_depth=None, min_samples_split=2, mode='entropy'):
    
    
    if max_depth == 0 or len(np.unique(y_train)) == 1 or len(y_train) < min_samples_split:
        return {'leaf': True, 'class': np.argmax(np.bincount(y_train))}
    
    samples, feat_index = X_train.shape
    all_features = X_train.shape[1]

    feature_indexes = np.random.choice(feat_index, all_features, replace=False)

    best_feature = None
    best_value = None
    best_gain = 0

    for feature_idx in feature_indexes:
        unique_values = np.unique(X_train[:, feature_idx])
        for limit in unique_values:
            gain = calculate_information_gain(X_train, y_train, feature_idx, limit, mode)
            if best_gain < gain:
                best_gain = gain
                best_feature = feature_idx
                best_value = limit
    
    if best_gain == 0:
        return {'leaf': True, 'class': np.argmax(np.bincount(y_train))}
    
    left_X, left_y = X_train[X_train[:, best_feature] <= best_value], y_train[X_train[:, best_feature] <= best_value ]
    right_X, right_y = X_train[X_train[:, best_feature] > best_value ], y_train[X_train[:, best_feature] > best_value ]
    
    left_tree = decision_tree(left_X, left_y,  max_depth-1, min_samples_split)
    right_tree = decision_tree(right_X, right_y, max_depth-1, min_samples_split)

    return {'leaf': False, 'left': left_tree, 'right': right_tree, 'feature': best_feature, 'limit': best_value}


##### Prediction Function

###### Traverses through each node to make predictions

In [498]:
def predict(tree, X):
    #fills in zero value for the entirety of samples
    predictions = np.ones(X.shape[0])

    for i in range(X.shape[0]):
        node = tree
        while not node['leaf']:
            feature_idx = node['feature']
            if X[i, feature_idx] <= node['limit']:
                node = node['left']
            else:
                node = node['right']
        predictions[i] = node['class']
    return predictions

##### Function to post-prune the decision tree

In [499]:
def post_prune(tree, X_test, y_test):
    
    if tree['leaf']:
        return tree

    
    left_tree = post_prune(tree['left'], X_test, y_test)
    right_tree = post_prune(tree['right'], X_test, y_test)

    
    if left_tree['leaf'] and right_tree['leaf']:
        # Calculate the error of the current node
        predictions = predict(tree, X_test)
        error = np.mean(predictions != y_test)

        left_predictions = predict(left_tree, X_test)
        left_error = np.mean(left_predictions != y_test)
        right_predictions = predict(right_tree, X_test)
        right_error = np.mean(right_predictions != y_test)

        
        if error - (left_error + right_error) / 2 < 0.01:
            return {'leaf': True, 'class': np.argmax(np.bincount(y_test))}

    
    return {'leaf': False, 'left': left_tree, 'right': right_tree, 'feature': tree['feature'], 'limit': tree['limit']}

In [500]:
tree = decision_tree(X_train, y_data, max_depth=5, mode='gini')

In [501]:
tree_pruned = post_prune(tree, X_test, y_test)

In [502]:
y_predictions = predict(tree, X_test)
y_predictions_afterpruning = predict(tree_pruned, X_test)
print(y_predictions)
print(y_predictions_afterpruning)


[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [503]:
def accuracy(y_test, predictions):
    return np.sum(y_test == predictions) / len(y_test)

In [504]:
print("Accuracy without pruning ",accuracy(y_test, y_predictions))
print("Accuracy with Pruning ",accuracy(y_test, y_predictions_afterpruning))

Accuracy without pruning  0.8421862971516552
Accuracy with Pruning  0.8344880677444187


## K fold cross validation

Creating train and test dataset from the data(wine-quality)

In [505]:
#taking k = 10
k = 10

data_X = data.drop('quality', axis=1).values
data_y = data['quality'].values.flatten()
data_y = np.where(data_y >= 7 , 1, 0)
print(data_X.shape)
print(data_y.shape)
print(data_y)


(6495, 11)
(6495,)
[0 1 1 ... 0 1 0]


Dividing the entire dataset in k sets, creating a list of lists(containing values for certain indexes)

In [506]:
sample_size, features = data_X.shape
new_size = round(sample_size/k)

data_Xi = list()
data_yi = list()


for i in range(k):
    if i == k-1:
        X = data_X[(k-1)*new_size: sample_size, :]
        y = data_y[(k-1)*new_size: sample_size]
        data_Xi.append(X.tolist())
        data_yi.append(y.tolist())


    else:
        X = data_X[ i*new_size : (i+1)*new_size , : ]
        y = data_y[ i*new_size : (i+1)*new_size ]
        data_Xi.append(X.tolist())
        data_yi.append(y.tolist())
        
 

##### Calling Decision tree for k fold cross validation

Creating X_train_new and y_train_new by merging k-1 datasets and calling decision tree for this new dataset 

Checking predictions for X_test_new using y_test_new

In [507]:
accuracy_sum = 0

X_trainf = []
X_testf = []
y_trainf = []
y_testf = []

for i in range(k):
    X_test = []
    y_test = []
    X_train = []
    y_train = []
    
    for j in range(k):
        if j == i:
            X_test.extend(data_Xi[j])
            y_test.extend(data_yi[j])
        else:
            X_train.extend(data_Xi[j])
            y_train.extend(data_yi[j])
    
    X_testf.append(X_test)
    y_testf.append(y_test)
    X_trainf.append(X_train)
    y_trainf.append(y_train)

    
for i in range(k):
    X_train_new = np.array(X_trainf[i])
    X_test_new = np.array(X_testf[i])
    y_train_new = np.array(y_trainf[i])
    y_test_new = np.array(y_testf[i])


    #Decision Tree Call
    tree = decision_tree(X_train_new, y_train_new, max_depth=5)
    predictions = predict(tree, X_test_new)
    data_acc = accuracy(y_test_new, predictions)
    print("Accuracy for the iteration",i, "is: ",data_acc)
    accuracy_sum += data_acc
    



Accuracy for the iteration 0 is:  0.7276923076923076
Accuracy for the iteration 1 is:  0.8
Accuracy for the iteration 2 is:  0.8092307692307692
Accuracy for the iteration 3 is:  0.8076923076923077
Accuracy for the iteration 4 is:  0.816923076923077
Accuracy for the iteration 5 is:  0.8076923076923077
Accuracy for the iteration 6 is:  0.8353846153846154
Accuracy for the iteration 7 is:  0.8876923076923077
Accuracy for the iteration 8 is:  0.8584615384615385
Accuracy for the iteration 9 is:  0.8217054263565892


In [508]:
print("Accuracy of the Decision Tree with k-fold Cross Validation with k=10:", accuracy_sum/k)

Accuracy of the Decision Tree with k-fold Cross Validation with k=10: 0.817247465712582
